# Linear regression

This notebook compares various linear regression implementations. The dataset used is the [Toulouse bike sharing dataset](https://creme-ml.github.io/generated/creme.datasets.fetch_bikes.html#creme.datasets.fetch_bikes).

In [28]:
%load_ext watermark
%watermark --python --machine --packages creme,keras,sklearn,tensorflow,torch --datename

Tue Aug 20 2019 

CPython 3.7.3
IPython 7.4.0

creme 0.3.0
keras 2.2.4
sklearn 0.20.3
tensorflow 1.14.0
torch 1.1.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.1.9-arch1-1-ARCH
machine    : x86_64
processor  : 
CPU cores  : 8
interpreter: 64bit


In [11]:
from creme import compose
from creme import datasets
from creme import feature_extraction
from creme import linear_model
from creme import metrics
from creme import optim
from creme import preprocessing
from creme import stats
from keras import layers
from keras import models
from keras import optimizers
from sklearn import linear_model as sk_linear_model
import torch

%run utils.py
%run wrappers.py

In [12]:
n_features = 6
lr = 0.005

class PyTorchNet(torch.nn.Module):
    
    def __init__(self, n_features):
        super().__init__()
        self.linear = torch.nn.Linear(n_features, 1)
        torch.nn.init.constant_(self.linear.weight, 0)
        torch.nn.init.constant_(self.linear.bias, 0)
        
    def forward(self, x):
        return self.linear(x)
    
torch_model = PyTorchNet(n_features=n_features)

# Keras
inputs = layers.Input(shape=(n_features,))
predictions = layers.Dense(1, kernel_initializer='zeros', bias_initializer='zeros')(inputs)
keras_model = models.Model(inputs=inputs, outputs=predictions)
keras_model.compile(optimizer=optimizers.SGD(lr=lr), loss='mean_squared_error')


def add_hour(x):
    x['hour'] = x['moment'].hour
    return x

results = benchmark(
    get_X_y=datasets.fetch_bikes,
    n=182470,
    get_pp=lambda: (
        compose.Whitelister('clouds', 'humidity', 'pressure', 'temperature', 'wind') +
        (
            add_hour |
            feature_extraction.TargetAgg(by=['station', 'hour'], how=stats.Mean())
        ) |
        preprocessing.StandardScaler()
    ),
    models=[
        ('creme', 'LinearRegression', linear_model.LinearRegression(
            optimizer=optim.SGD(lr),
            l2=0.,
            intercept_lr=lr
        )),
        ('scikit-learn', 'SGDRegressor', ScikitLearnRegressor(
            model=sk_linear_model.SGDRegressor(
                learning_rate='constant',
                eta0=lr,
                penalty='none'
            ),
        )),
        ('PyTorch (CPU)', 'Linear', PyTorchRegressor(
            network=torch_model,
            loss_fn=torch.nn.MSELoss(),
            optimizer=torch.optim.SGD(torch_model.parameters(), lr=lr)
        )),
        ('Keras on Tensorflow (CPU)', 'Dense', KerasRegressor(
            model=keras_model
        )),
    ],
    get_metric=metrics.MSE
)

W0820 13:54:40.275430 139959188182848 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0820 13:54:40.280425 139959188182848 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0820 13:54:40.281914 139959188182848 deprecation_wrapper.py:119] From /home/max/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



In [26]:
results

,Library,Model,MSE,Fit time,Average fit time,Predict time,Average predict time
0,creme,LinearRegression,23.035085,"2s, 961ms, 90μs, 750ns","16μs, 228ns","932ms, 944μs, 854ns","5μs, 113ns"
1,scikit-learn,SGD,25.295369,"43s, 78ms, 345μs, 590ns","236μs, 85ns","17s, 515ms, 602μs, 700ns","95μs, 992ns"
2,PyTorch (CPU),Linear,23.035086,"2m, 7s, 300ms, 182μs, 229ns","697μs, 650ns","43s, 140ms, 822μs, 95ns","236μs, 427ns"
3,Keras on Tensorflow (CPU),Linear,23.035086,"4m, 4s, 177ms, 471μs, 22ns","1ms, 338μs, 179ns","2m, 28s, 629ms, 461μs, 634ns","814μs, 542ns"
